# Lab 4: Mindspore实现手写数字识别

## Before we start

我们创建了[实验课的github仓库](https://github.com/Yujie-G/ML-2024Spring)，你可以在这里找到所有的实验指导书和相关资源。

由于众所周知的原因，我们会在智慧树平台上上传一份实验资源的拷贝，不使用git仓库**不会**影响你完成实验。

为什么使用git?

1. 你可以第一时间获取实验指导代码的更新，代码框架的修改等。
2. 你可以方便的在本地查看代码的变更和历史。
3. 你可以在issue中提出关于实验代码的问题，可以帮助到有相同问题的同学。


How to start:

初始化：
```bash
git clone git@github.com:Yujie-G/ML-2024Spring.git
```

之后，每次新实验发布，你可以通过以下命令来更新本地仓库：
```bash
git pull
```


## TODO

1. 安装Mindspore
2. 阅读并理解全连接网络的实现代码
3. 实现LeNet5网络(部分代码已给出)

## 1. 安装Mindspore

[进入官网获取下载命令](https://www.mindspore.cn/install)， 建议选择2.1.1版本的Mindspore, 安装方式选择pip/conda安装均可

不会Mindspore?可以点击[这里](https://www.mindspore.cn/tutorials/zh-CN/r2.2/index.html)学习官方教程

你也可以参考[官方的API文档](https://www.mindspore.cn/docs/zh-CN/r2.1/index.html)

## 2. 利用Mindspore实现全连接网络手写数字识别

你可以参考[这份华为官方的指导手册](./assets/MindsporeTutorial.pdf),查看Mindspore的教程

In [1]:
import mindspore
from mindspore import ops
from mindspore import nn
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# 加载MNIST数据集
train_dataset_dir = "./MNIST/train"
train_dataset = MnistDataset(dataset_dir=train_dataset_dir)
test_dataset_dir = "./MNIST/test"
test_dataset = MnistDataset(dataset_dir=test_dataset_dir)

print(train_dataset.get_col_names())

['image', 'label']


In [3]:
def datapipe(dataset, batch_size):
    image_transforms = [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW()
    ]
    label_transform = transforms.TypeCast(mindspore.int32)

    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = datapipe(train_dataset, 64)
test_dataset = datapipe(test_dataset, 64)

In [4]:
for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break

Shape of image [N, C, H, W]: (64, 1, 28, 28) Float32
Shape of label: (64,) Int32


In [5]:
for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break

Shape of image [N, C, H, W]: (64, 1, 28, 28) Float32
Shape of label: (64,) Int32


In [6]:
# 网络构建
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )
    
    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits
    
model = Network()
print(model)

Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


In [7]:
# 模型训练
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), 1e-2)#优化器 梯度下降

In [8]:
def train(model, dataset, loss_fn, optimizer):
    def forward_fn(data, label):#计算模型前向传播的结果和损失
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits
    #计算损失函数关于模型参数的梯度
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)
    
    def train_step(data, label):#执行一步训练
        (loss, _), grads = grad_fn(data, label)#调用grad_fn计算损失和梯度
        loss = ops.depend(loss, optimizer(grads))#使用优化器更新模型参数
        return loss
    
    size = dataset.get_dataset_size()#获取数据集的大小
    model.set_train()
    #迭代训练数据集，每次从数据集中获取一个批次的数据和标签
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)#调用train_step函数执行一步训练，并获取计算得到的损失值
        
        if batch % 100 == 0:#每经过100个批次，输出一次当前的损失值和训练进度信息
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

In [9]:
def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()#获取数据集的批次数量
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches#计算平均测试损失
    correct /= total#准确率
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 3#周期数
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset, loss_fn, optimizer)
    test(model, test_dataset, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.294447  [  0/938]
loss: 1.637735  [100/938]
loss: 0.846680  [200/938]
loss: 0.493651  [300/938]
loss: 0.438014  [400/938]
loss: 0.677162  [500/938]
loss: 0.585079  [600/938]
loss: 0.352135  [700/938]
loss: 0.598571  [800/938]
loss: 0.289243  [900/938]
Test: 
 Accuracy: 90.6%, Avg loss: 0.322850 

Epoch 2
-------------------------------
loss: 0.196607  [  0/938]
loss: 0.590457  [100/938]
loss: 0.320770  [200/938]
loss: 0.189132  [300/938]
loss: 0.474729  [400/938]
loss: 0.292380  [500/938]
loss: 0.353366  [600/938]
loss: 0.351633  [700/938]


In [11]:
# Save checkpoint 保存模型的检查点
mindspore.save_checkpoint(model, "model.ckpt")
print("Saved Model to model.ckpt")

Saved Model to model.ckpt


In [12]:
# Instantiate a random initialized model
model = Network()
# Load checkpoint and load parameter to model
param_dict = mindspore.load_checkpoint("model.ckpt")
param_not_load, _ = mindspore.load_param_into_net(model, param_dict)
print(param_not_load)

[]


In [13]:
model.set_train(False)
for data, label in test_dataset:
    pred = model(data)
    predicted = pred.argmax(1)
    print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
    break

Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >

Predicted: "[8 7 2 9 9 1 4 6 9 4]", Actual: "[3 2 2 9 9 1 4 6 9 4]"


## 3. 实现LeNet5网络

![](assets/images/2024-04-07-22-36-11.png)


根据上图说明的参数，实现LeNet5网络，完成手写数字识别任务，部分代码已给出，你需要补全代码

你可能会用到的[MindSpore卷积神经网络API](https://www.mindspore.cn/docs/zh-CN/r2.1/api_python/mindspore.nn.html#%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C%E5%B1%82)

In [1]:
# import necessary pkgs
import os
import numpy as np
import mindspore as ms
import mindspore.nn as nn
from mindspore import Model
import mindspore.dataset as ds

import mindspore
from mindspore import ops
from mindspore import nn
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# !设置全局种子，这里改成你的学号后四位
np.random.seed(1030)
ms.set_seed(1030)


## hyperparameters 超参数
batch_size = 32
epoch_size = 10
learning_rate = 0.01

dataset_dir = "./MNIST"


In [3]:
# 加载MNIST数据集
train_dataset_dir = "./MNIST/train"
train_dataset = MnistDataset(dataset_dir=train_dataset_dir)
test_dataset_dir = "./MNIST/test"
test_dataset = MnistDataset(dataset_dir=test_dataset_dir)

print(train_dataset.get_col_names())

for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break

['image', 'label']
Shape of image [N, C, H, W]: (28, 28, 1) UInt8
Shape of label: () UInt32


In [4]:
def create_dataset(data_path, batch_size=32, repeat_size=1, num_parallel_workers=1):

    # 创建数据集
    mnist_ds = ds.MnistDataset(data_path)

    # 实现数据增强和处理
    # 1. 将图像的对比度和亮度做适当的调整，调整幅度任意。
    # 2. 将图像缩放到模型需要的输入，比如32x32。
    
    image_transform = [
   
    vision.Rescale(1.0 / 255.0, 0),
    vision.Normalize(mean=(0.1307,), std=(0.3081,)),
    vision.HWC2CHW(),
        
        
    #mindspore.dataset.vision.RandomAutoContrast(),  # 调整对比度
    #mindspore.dataset.vision.AdjustBrightness(1.5),  # 调整亮度
    mindspore.dataset.vision.Resize([32,32]),  # 缩放图像到32x32大小
    #mindspore.dataset.vision.Resize(32) 
    ]
    label_transform = transforms.TypeCast(mindspore.int32)
    
    
    
    

    # 修改这行代码，使得你的增强操作生效
    #mnist_ds = mnist_ds.map(operations=transforms,input_columns=["label", "image"], num_parallel_workers=num_parallel_workers)
    mnist_ds=mnist_ds.map(image_transform,"image")
    mnist_ds=mnist_ds.map(label_transform,"label")

    # 处理生成的数据集
    buffer_size = 10000
    mnist_ds = mnist_ds.shuffle(buffer_size=buffer_size)
    mnist_ds = mnist_ds.batch(batch_size, drop_remainder=True)
    mnist_ds = mnist_ds.repeat(repeat_size)

    return mnist_ds

train_dataset = create_dataset(os.path.join(dataset_dir, "train"), batch_size=batch_size)
test_dataset = create_dataset(os.path.join(dataset_dir, "test"), batch_size=batch_size)


image, label = next(train_dataset.create_tuple_iterator())

In [5]:
for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break

Shape of image [N, C, H, W]: (32, 32, 32, 28) Float32
Shape of label: (32,) Int32


In [18]:
class LeNet5(nn.Cell):

    def __init__(self):
        super().__init__()
        #self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            mindspore.nn.Conv2d(32,6*32,5),
            mindspore.nn.ReLU(),
            mindspore.nn.MaxPool2d(2,2),
            mindspore.nn.ReLU(),
            mindspore.nn.Conv2d(6,16,5),
            mindspore.nn.ReLU(),
            mindspore.nn.MaxPool2d(2,2),
            mindspore.nn.ReLU(),
            nn.Flatten(),
            nn.Dense(16*5*5, 120),
            nn.ReLU(),
            nn.Dense(120, 84),
            nn.ReLU(),
            nn.Dense(84, 10)          
        )
    
    def construct(self, x):   
        logits = self.dense_relu_sequential(x)
        return logits
    
    
#model = LeNet5()
#print(model)    

LeNet5<
  (dense_relu_sequential): SequentialCell<
    (0): Conv2d<input_channels=1, output_channels=6, kernel_size=(5, 5), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=<mindspore.common.initializer.HeUniform object at 0x000002740FA76700>, bias_init=None, format=NCHW>
    (1): ReLU<>
    (2): MaxPool2d<kernel_size=2, stride=2, pad_mode=VALID>
    (3): ReLU<>
    (4): Conv2d<input_channels=6, output_channels=16, kernel_size=(5, 5), stride=(1, 1), pad_mode=same, padding=0, dilation=(1, 1), group=1, has_bias=False, weight_init=<mindspore.common.initializer.HeUniform object at 0x000002740D6B21F0>, bias_init=None, format=NCHW>
    (5): ReLU<>
    (6): MaxPool2d<kernel_size=2, stride=2, pad_mode=VALID>
    (7): ReLU<>
    (8): Flatten<>
    (9): Dense<input_channels=400, output_channels=120, has_bias=True>
    (10): ReLU<>
    (11): Dense<input_channels=120, output_channels=84, has_bias=True>
    (12): ReLU<>
    (13): Dense<input_channels=84

In [19]:
net = LeNet5()
loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction='mean')
optim = nn.Momentum(params=net.trainable_params(), learning_rate=learning_rate, momentum=0.9)
#model = Model(network = net, loss_fn=loss, optimizer=optim, metrics={"Accuracy": nn.Accuracy()})

In [20]:
# 实现训练部分代码，并打印训练过程中的loss值，建议可视化查看loss值的变化

def Train(model, dataset, loss_fn, optimizer):
    def forward_fn(data, label):#计算模型前向传播的结果和损失
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits
    #计算损失函数关于模型参数的梯度
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)
    
    def train_step(data, label):#执行一步训练
        (loss, _), grads = grad_fn(data, label)#调用grad_fn计算损失和梯度
        loss = ops.depend(loss, optimizer(grads))#使用优化器更新模型参数
        return loss
    
    size = dataset.get_dataset_size()#获取数据集的大小
    model.set_train()
    #迭代训练数据集，每次从数据集中获取一个批次的数据和标签
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)#调用train_step函数执行一步训练，并获取计算得到的损失值
        
        if batch % 100 == 0:#每经过100个批次，输出一次当前的损失值和训练进度信息
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")
            
            
            
def Test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()#获取数据集的批次数量
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches#计算平均测试损失
    correct /= total#准确率
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [22]:
epochs = 3#周期数
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    Train(model, train_dataset, loss, optim)
    Test(model, test_dataset, loss)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: For 'Conv2D', 'C_in' of input 'x' shape divide by parameter 'group' must be equal to 'C_in' of input 'weight' shape: 1, but got 'C_in' of input 'x' shape: 32, and 'group': 1.

----------------------------------------------------
- C++ Call Stack: (For framework developers)
----------------------------------------------------
mindspore\core\ops\conv2d.cc:240 mindspore::ops::`anonymous-namespace'::Conv2dInferShape


In [40]:
## test
def test_net(network, model, path):
    """Define the evaluation method."""
    # 加载已保存的模型
    param_dict = ms.load_checkpoint(path)
    # load parameter to the network
    ms.load_param_into_net(network, param_dict)
    # evaluation
    acc = model.eval(test_dataset, dataset_sink_mode=False)
    print("============== Accuracy:{} ==============".format(acc))

# 修改为你的checkpoint路径
test_net(net, model, "model.ckpt")

ValueError: For 'load_checkpoint', the checkpoint file: E:\python\JupyterNotebook\Lab4-LeNet\model.ckpt does not exist, please check whether the 'ckpt_file_name' is correct.